In [ ]:
from dotenv import load_dotenv 

load_dotenv()

True

In [ ]:
from langchain_groq import ChatGroq 

model = ChatGroq(model_name="meta-llama/llama-4-scout-17b-16e-instruct")

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_prompt = """
Give 3 Code Test Questions.
You are a code reviewer.
- You have to give the code test based on user level.
- You have to give the user to code test which can be all programming languages but you dont have to MENTION.
After that give them a feedback.
- Check their logic step-by-step.
- If the code is incorrect or incomplete, explain where it breaks and how to fix it.
- If the code is mostly correct, provide helpful suggestions to improve.
- Be friendly and constructive.
Say goodbyes.
"""

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ]
)

runnable = prompt_template | model


In [12]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import uuid


store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [13]:
chain = prompt_template | model
session_id = str(uuid.uuid4())

In [17]:
with_message_history.invoke(
    {"input": """def check_voting_eligibility():
    try:
        age = int(input("Enter your age: "))
        if age >= 18:
            print("You are eligible to vote.")
        else:
            print("You are not eligible to vote yet.")
    except ValueError:
        print("Please enter a valid number for age.")

check_voting_eligibility()
"""},
    config={"configurable": {"session_id": session_id}},
).content

'**Code Review**\n\nYour code looks clean and straightforward. Here\'s a breakdown of what you\'re doing:\n\n1. You\'re defining a function `check_voting_eligibility` that takes no arguments.\n2. You\'re using a `try`-`except` block to catch any `ValueError` exceptions that might occur when trying to convert the user\'s input to an integer.\n3. You\'re prompting the user to enter their age and storing it in the `age` variable.\n4. You\'re checking if the user\'s age is greater than or equal to 18 and printing out a message indicating whether they\'re eligible to vote or not.\n\n**Feedback**\n\n* Your code is well-structured and easy to read.\n* You\'re handling potential errors by catching `ValueError` exceptions.\n* Your function name is descriptive and indicates what the function does.\n\n**Suggestions**\n\n* Consider adding a docstring to your function to describe what it does, what inputs it expects, and what outputs it produces.\n* You might want to consider adding a loop to allow

'**Code Review**\n\nYour code looks great for a beginner! Here\'s a step-by-step review:\n\n### Logic\n\n1. You\'re taking two numbers as input from the user using `input()` and converting them to integers using `int()`. This is correct!\n2. You\'re storing the inputs in variables `a` and `b`, which is a good practice.\n3. You\'re calculating the sum of `a` and `b` using the `+` operator, which is correct.\n4. You\'re printing the result using `print()`, which is correct.\n\n### Code Quality\n\n* Your code is concise and easy to read.\n* You\'re using meaningful variable names (`a` and `b` could be more descriptive, but it\'s okay for a beginner).\n* You\'re handling the input conversion to integers correctly.\n\n### Suggestions for Improvement\n\n* Consider adding a prompt to inform the user what the output represents (e.g., "The sum is:").\n* You could use more descriptive variable names, such as `num1` and `num2`.\n* You might want to add some error handling to handle cases where th